In [12]:
import pandas as pd
from constraint import Problem, AllDifferentConstraint

In [13]:
muscle_map_df = pd.read_csv('muscle_map.csv')
# I want to get all the distinct values from column Body Part
muscle_map_df['Muscle'] = muscle_map_df['Muscle'].apply(lambda x: x.lower())
# rename values with Thigs with Thighs
muscle_map_df['Body Part'] = muscle_map_df['Body Part'].apply(lambda x: x.replace('Thigs', 'Thighs'))
muscle_map_df['Body Part'] = muscle_map_df['Body Part'].apply(lambda x: x.replace('neck', 'Neck'))
muscle_map_df.head()

,Muscle,Body Part
0,hamstrings (below knees),Calves
1,"erector spinae, cervicis & capitis fibers",Back
2,triceps brachii,Upper Arms
3,gracilis,Thighs
4,"pectoralis major, sternal",Chest


In [14]:
muscle_map_df['Body Part'].unique()

array(['Calves', 'Back', 'Upper Arms', 'Thighs', 'Chest', 'Hips',
       'Forearms', 'Shoulders', 'Waist', 'Neck'], dtype=object)

In [35]:
def useless_replacement(x):
     x = x.lower()
     return x.replace("none", "").replace("no significant stabilizers", "").replace("(see comments)", "").replace("(neck)","").replace("see comments", "").title()

In [36]:
exercises_df = pd.read_csv('exrx.csv')
# Transform all the target muscles into a single string in lower case. It is a string array by default
exercises_df['target_muscles'] = exercises_df['target_muscles'].apply(lambda x: x[2:-2].lower())
# # Convert all muscle names to lowercase in the mapping dictionary
muscle_map_dict = dict(zip(muscle_map_df['Muscle'], muscle_map_df['Body Part']))

# Use map and lambda to create the 'major muscle' column in 'exercises_df'
exercises_df['minor_muscle'] = exercises_df['target_muscles'].apply(lambda x: muscle_map_dict.get(x, 'Unknown'))
#remove the rows with 'Unknown' major muscle
exercises_df = exercises_df[exercises_df['minor_muscle'] != 'Unknown']
#remove the rows with 'Specialized' utility
exercises_df = exercises_df[exercises_df['utility'] != 'Specialized']
#remove the rows with Null utility
exercises_df = exercises_df[exercises_df['utility'].notnull()]
# Make a dict that has Legs , Chest, Back, Arms and Shoulders as keys and the values are the exercises that belong to that muscle group
muscle_group_dict = {'Legs':['Thighs','Hips','Calves','Waist'] , 'Chest':['Chest'], 'Back':['Back'], 'Arms':['Forearms','Upper Arms'] ,'Shoulders':['Neck','Shoulders']}
# Make a column major_muscle that is based on the value from minor_muscle column and use the dict to map it to the major muscle group
exercises_df['major_muscle'] = exercises_df['minor_muscle'].apply(lambda x: [k for k,v in muscle_group_dict.items() if x in v][0])
# Remove rows with duplicate exercise names
exercises_df = exercises_df.drop_duplicates(subset=['exercise'])
exercises_df = exercises_df.drop(columns=['force'])
# replace None string with "" in stablizer_muscles column
exercises_df['stabilizer_muscles'] = exercises_df['stabilizer_muscles'].apply(lambda x: useless_replacement(x))
# Repeat the same process as above for columns synergist_muscles,dynamic_stabilizer_muscles,antagonist_stabilizer_muscles
exercises_df['synergist_muscles'] = exercises_df['synergist_muscles'].apply(lambda x: useless_replacement(x))
exercises_df['dynamic_stabilizer_muscles'] = exercises_df['dynamic_stabilizer_muscles'].apply(lambda x: useless_replacement(x))
exercises_df['antagonist_stabilizer_muscles'] = exercises_df['antagonist_stabilizer_muscles'].apply(lambda x: useless_replacement(x))

exercises_df['utility'] = exercises_df['utility'].apply(lambda x: 'Basic or Auxiliary' if x == 'Auxiliary or Basic' else x)
exercises_df.head()

,exercise,utility,mechanics,target_muscles,synergist_muscles,stabilizer_muscles,dynamic_stabilizer_muscles,antagonist_stabilizer_muscles,minor_muscle,major_muscle
0,Barbell Triceps Extension,Basic or Auxiliary,Isolated,triceps brachii,[''],"['Deltoid, Anterior', 'Pectoralis Major, Clavi...",[''],[''],Upper Arms,Arms
1,Cable Bar Lying Triceps Extension,Basic,Isolated,triceps brachii,[''],"['Deltoid, Anterior', 'Pectoralis Major, Clavi...",[''],[''],Upper Arms,Arms
2,Smith Rear Delt Row,Basic or Auxiliary,Compound,"deltoid, posterior","['Infraspinatus', 'Teres Minor', 'Deltoid, Lat...","['Erector Spinae', 'Hamstrings', 'Gluteus Maxi...",['Biceps Brachii'],"['Rectus Abdominis', 'Obliques']",Shoulders,Shoulders
3,Lever Hack Squat (plate loaded),Basic or Auxiliary,Compound,gluteus maximus,"['Quadriceps', 'Adductor Magnus', 'Soleus']",[''],"['Hamstrings', 'Gastrocnemius']",[''],Hips,Legs
4,Sled Lying Reverse Calf Press (plate loaded),Basic or Auxiliary,Isolated,tibialis anterior,[''],[''],[''],[''],Calves,Legs


In [37]:
# find rows that have compound mechanics and auxiliary utility
exercises_df[(exercises_df['mechanics'] == 'Compound') & (exercises_df['utility'] == 'Auxiliary')]

,exercise,utility,mechanics,target_muscles,synergist_muscles,stabilizer_muscles,dynamic_stabilizer_muscles,antagonist_stabilizer_muscles,minor_muscle,major_muscle
17,Barbell Lateral Step-up,Auxiliary,Compound,quadriceps,"['Gluteus Maximus', 'Adductor Magnus', 'Soleus...","['Erector Spinae', 'Gluteus Medius', 'Gluteus ...","['Hamstrings', 'Gastrocnemius (Lead Leg)']","['Rectus Abdominis', 'Obliques']",Thighs,Legs
27,Suspended Power Pull,Auxiliary,Compound,"back, general","['Trapezius, Middle', 'Trapezius, Lower', 'Rho...","['Erector Spinae', 'Gluteus Maximus', 'Hamstri...","['Biceps Brachii', 'Triceps, Long Head']",['Rectus Abdominis'],Back,Back
28,Cable Split Squat (belt),Auxiliary,Compound,gluteus maximus,"['Quadriceps', 'Adductor Magnus', 'Soleus']",[''],"['Hamstrings', 'Gastrocnemius']",[''],Hips,Legs
53,Barbell Walking Lunge,Auxiliary,Compound,gluteus maximus,"['Quadriceps', 'Adductor Magnus', 'Soleus']","['Erector Spinae', 'Tibialis Anterior', 'Glute...","['Hamstrings', 'Gastrocnemius']",[''],Hips,Legs
54,Barbell Side Split Squat,Auxiliary,Compound,gluteus maximus,"['Quadriceps', 'Adductor Magnus (Lead Leg)', '...","['Erector Spinae', 'Gluteus Medius', 'Gluteus ...","['Hamstrings', 'Gastrocnemius']",[''],Hips,Legs
...,...,...,...,...,...,...,...,...,...,...
1279,Cable One Arm Chest Press,Auxiliary,Compound,"pectoralis major, sternal","['Pectoralis Major, Clavicular', 'Deltoid, Ant...",[' '],"['Biceps Brachii, Short Head']",[''],Chest,Chest
1320,Barbell Board Press,Auxiliary,Compound,"pectoralis major, sternal","['Pectoralis Major, Clavicular', 'Deltoid, Ant...","['Erector Spinae', 'Gluteus Maximus']","['Biceps Brachii, Short Head']",[''],Chest,Chest
1376,Cable One Arm Standing Incline Chest Press,Auxiliary,Compound,"pectoralis major, clavicular","['Pectoralis Major, Sternal', 'Deltoid, Anteri...",['Rectus Abdominis'],"['Biceps Brachii, Short Head']",['Erector Spinae'],Chest,Chest
1397,Lever Single Leg Standing Leg Press (plate loa...,Auxiliary,Compound,quadriceps,"['Gluteus Maximus', 'Adductor Magnus']","['Erector Spinae', 'Obliques', 'Gluteus Medius...",[''],"['Hamstrings', 'Rectus Abdominis']",Thighs,Legs


In [38]:
# output the dataframe to a csv file
exercises_df.to_csv('exercises_cleaned.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'exercises_cleaned.csv'